In [1]:
import requests
import zipfile
import io
import ssl
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import scipy.stats as ss
from scipy.stats import t
from scipy.io.arff import loadarff
from os import path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

#open .arff file if it exists in directory or fetch it from the web
arff_file = '5year.arff'

if path.exists(arff_file) == False:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00365/data.zip'
    r = requests.get(url)
    textfile = zipfile.ZipFile(io.BytesIO(r.content))
    textfile.extract(arff_file)

file = open(arff_file, 'r')
filedata = file.read()
filedata = filedata.replace('class {0,1}','Attr65 numeric')

file = open(arff_file, 'w')
file.write(filedata)
file.close()

#Convert .arff file to a dataframe
data = loadarff(arff_file)
df = pd.DataFrame(data[0])

# Show relavant statistics
allStats = df.describe(include='all')

# Show relevant statistics with outliers removed
df_NO = df.loc[:,'Attr1':'Attr64']
df_NO = df_NO[(df_NO >= df_NO.mean()-2*df_NO.std()) &
                        (df_NO <= df_NO.mean()+2*df_NO.std())]
df_NO['Target'] = df['Attr65']
allStats_NO = df_NO.describe(include='all')

#Fill all missing values with the mean (df1)
df1 = df_NO.fillna(df_NO.mean())

#Remove rows with any Nan values (df2)
df2 = df_NO.dropna().reset_index(drop=True)

#Scaling of predictors
df_scale = df1
df3 = df_scale.drop('Target', axis=1)
df3_scaler = StandardScaler()
df3a = df3_scaler.fit_transform(df3.values)
df3 = pd.DataFrame(df3a, columns=df3.columns)
df3 = df3.join(df_scale.Target)

# Choose new dataframe to be df1, df2, or df3 (scaled)
df_NO = df1

#Show correlation matrix to see if Attr37 is highly correlated with anything
corrMat = df_NO.corr()
corrMat['Target'].nsmallest(1), corrMat['Target'].nlargest(2)

# Create a dataframe of attributes and their meanings
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data'
url_get = requests.get(url)
soup = BeautifulSoup(url_get.content, 'lxml')
with open('attributes.txt', 'w', encoding='utf-8') as f_out:
    f_out.write(soup.prettify())
    f_out.close()

with open("attributes.txt", "r") as f:
    lines = f.readlines()
    lines = lines[367:494]
    f.close()
    
finance_def_df = pd.DataFrame({'expression':lines})
finance_def_df = finance_def_df[finance_def_df.index%2 == 0]
finance_def_df = finance_def_df.replace('^\s{6}X[0-9]+\t','',regex=True)\
.replace('$\n','',regex=True)
finance_def_df.index = df.loc[:,'Attr1':'Attr64'].columns

# Create DataFrame of strong correlations (negative and positive) based on correlation threshold
strongCorrMatrix = corrMat.unstack().reset_index()
strongCorrMatrix.rename(columns={'level_0':'Pair1',
                                 'level_1':'Pair2',0:'Correlation'}, inplace=True)
corrThresh = 0.95
strongCorrMatrix = strongCorrMatrix[((strongCorrMatrix['Correlation'] >= corrThresh) |
        (strongCorrMatrix['Correlation'] <= -corrThresh)) &
        (strongCorrMatrix['Pair1'] != strongCorrMatrix['Pair2'])]
strongCorrMatrix.reset_index(drop=True, inplace=True)
strongCorrMatrix = strongCorrMatrix.drop_duplicates(subset='Correlation', keep='first')

In [ ]:
#Investigate Attr7, Attr14, and Attr18
same_column_check = df1[['Attr7','Attr14', 'Attr18']]
attr7_14_slope, attr7_14_incept = np.polyfit(df_NO.Attr7, df_NO.Attr14, 1)
attr7_18_slope, attr7_18_incept = np.polyfit(df_NO.Attr7, df_NO.Attr18, 1)
attr14_18_slope, attr14_18_incept = np.polyfit(df_NO.Attr14, df_NO.Attr18, 1)

plt.plot(df_NO.Attr7, df_NO.Attr14, marker='.', linestyle='none', color='blue', label='true data')
plt.plot(df_NO.Attr7, attr7_14_incept+df_NO.Attr7*attr7_14_slope, color='green', label='regression line')
plt.legend()
plt.xlabel('Attr7')
plt.ylabel('Attr14')
plt.title('Regression and True Data of Attr7 vs Attr14')
plt.savefig('Attr7_vs_Attr14.jpg')

plt.plot(df_NO.Attr7, df_NO.Attr18, marker='.', linestyle='none', color='blue', label='true data')
plt.plot(df_NO.Attr7, attr7_18_incept+df_NO.Attr7*attr7_18_slope, color='green', label='regression line')
plt.legend()
plt.xlabel('Attr7')
plt.ylabel('Attr18')
plt.title('Regression and True Data of Attr7 vs Attr18')
plt.savefig('Attr7_vs_Attr18.jpg')

plt.plot(df_NO.Attr14, df_NO.Attr18, marker='.', linestyle='none', color='blue', label='true data')
plt.plot(df_NO.Attr14, attr14_18_incept+df_NO.Attr14*attr14_18_slope, color='green', label='regression line')
plt.legend()
plt.xlabel('Attr14')
plt.ylabel('Attr18')
plt.title('Regression and True Data of Attr14 vs Attr18')
plt.savefig('Attr14_vs_Attr18.jpg')

same_column = pd.DataFrame({'Definition':[finance_def_df.loc[i,'expression']\
        for i in same_column_check.columns[0:3]]}, index=same_column_check.columns[0:3])

In [ ]:
#Plot histogram of correlations for Target
sns.distplot(corrMat['Target'][corrMat['Target'] != 1],bins=10,kde=False,norm_hist=False, color='red')
plt.ylim(0,25)
plt.xlabel('Correlation')
plt.ylabel('Frequency of Occurrence')
plt.title('2012 Correlations of Attributes')
plt.savefig('Correlations_Target_2012.jpg')

In [ ]:
#Top 10 most corelated attributes with Target
top_ten = corrMat[['Target']][(corrMat['Target']) != 1]
top_ten['mag'] = abs(top_ten.Target)
top_ten = top_ten.nlargest(10, columns=['mag'])
top_ten['neg'] = np.where(top_ten.Target < 0, top_ten.mag, 0)
top_ten['pos'] = np.where(top_ten.Target > 0, top_ten.mag, 0)
top_ten = top_ten.sort_values('mag', ascending=True)

plt.barh(top_ten.index, top_ten.neg, color='red', label='negative correlation')
plt.barh(top_ten.index, top_ten.pos, color='blue', label='positive correlation')
plt.legend()
plt.grid(axis='x')
plt.xlim(0,0.35)
plt.xlabel('Correlation Magnitude')
plt.ylabel('Attribute')
plt.title('Correlation Magnitudes by Attributes')
plt.savefig('Correlations_magnitudes_Bar_2012.png')

In [ ]:
#Dataframe of Top 10 most corelated attributes with Target
top_ten_att = pd.DataFrame({'Definition':[finance_def_df.loc[i,'expression']\
                                          for i in top_ten.index]}, index=top_ten.index)

In [ ]:
#Average bankrupt companies over past 5 years
bankrupt_companies = np.array([271,400,495,515,410])
total_companies = np.array([7027,10173,10503,9792,5910])
bankrupt_percentage = bankrupt_companies/total_companies*100
sns.barplot([2008,2009,2010,2011,2012],bankrupt_percentage, color='red')
plt.grid(axis='y')
plt.ylim(0,8)
plt.xlabel('Year')
plt.ylabel('Percent (%)')
plt.title('Polish Bankruptcy Percentage by Year')
plt.savefig('Bankruptcy_percentage_year.jpg')

In [ ]:
# Create histograms of data
plt.subplots(figsize=(15, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.4)
for index in range(0,16):
    plt.subplot(4,4,index+1)
    df_NO.iloc[:,index].hist(bins=100)
    plt.xlabel('Values')
    plt.ylabel('Occurences')
    plt.ylim(top=df_NO.iloc[:,index].value_counts(bins=100).max())
    plt.legend((df_NO.columns[index],), loc=0)

plt.subplots(figsize=(15, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.4)
for index in range(16,32):
    plt.subplot(4,4,index-15)
    df_NO.iloc[:,index].hist(bins=100)
    plt.xlabel('Values')
    plt.ylabel('Occurences')
    plt.ylim(top=df_NO.iloc[:,index].value_counts(bins=100).max())
    plt.legend((df_NO.columns[index],), loc=0)

plt.subplots(figsize=(15, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.4)
for index in range(32,48):
    plt.subplot(4,4,index-31)
    df_NO.iloc[:,index].hist(bins=100)
    plt.xlabel('Values')
    plt.ylabel('Occurences')
    plt.ylim(top=df_NO.iloc[:,index].value_counts(bins=100).max())
    plt.legend((df_NO.columns[index],), loc=0)

plt.subplots(figsize=(15, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.4)
for index in range(48,64):
    plt.subplot(4,4,index-47)
    df_NO.iloc[:,index].hist(bins=100)
    plt.xlabel('Values')
    plt.ylabel('Occurences')
    plt.ylim(top=df_NO.iloc[:,index].value_counts(bins=100).max())
    plt.legend((df_NO.columns[index],), loc=0)

In [ ]:
#Hypothesis Testing

#1 - H0: mean(Attr2) + mean(Attr10) = 1 

mean_attr2 = df_NO.Attr2.mean()
mean_attr10 = df_NO.Attr10.mean()

sum_mean = mean_attr2 + mean_attr10
CI_mean_attr2 = t.interval(0.95, len(df_NO.Attr2)-1,\
                           loc=mean_attr2, scale=ss.sem(df_NO.Attr2))
CI_mean_attr10 = t.interval(0.95, len(df_NO.Attr10)-1,\
                           loc=mean_attr2, scale=ss.sem(df_NO.Attr10))

# Permutations are ONLY for unequal array samples
#This is just one example where we are summing the mean
def permutation_replicates(array1, array2, size):
    """Generate multiple permutation replicates."""

    array = np.concatenate((array1, array2))
    
    perm_sum_mean = np.empty(size)
    
    for i in range(size):
        perm_array = np.random.permutation(array)
        perm_array1 = perm_array[:len(array1)]
        perm_array2 = perm_array[len(array1):]
        perm_sum_mean[i] = np.mean(perm_array1) + np.mean(perm_array2)
        
    return perm_sum_mean

sum_mean_perm = permutation_replicates(df_NO.Attr2, df_NO.Attr10, 1000)

In [ ]:
#More Hypothesis Testing

#2
sum_means = np.mean(df_NO.Attr2) + np.mean(df_NO.Attr10)

def bootstrap_replicates(array1, array2, size):
    """Build bootstrap replicates for two arrays"""
    
    bs_sum_mean = np.empty(size)
        
    for i in range(size):
        bs_array1 = np.random.choice(array1)
        bs_array2 = np.random.choice(array2)
        bs_sum_mean[i] = np.mean(bs_array1) + np.mean(bs_array2)
        
    return bs_sum_mean

bs_sum_means = bootstrap_replicates(df_NO.Attr2, df_NO.Attr10, 10000)
plt.hist(bs_sum_means, bins=100)

p_value = np.sum(bs_sum_means <= sum_means)/len(bs_sum_means)

In [ ]:
#More Hypothesis Testing

#3
attr39_0 = df_NO.Attr39[df_NO.Target == 0]
attr39_1 = df_NO.Attr39[df_NO.Target == 1]

_ = plt.hist(attr39_0, bins=30)
_ = plt.axvline(x=np.mean(attr39_0), color='black', linestyle='dashed')
_ = plt.text(-0.65,1760,'Mean = {:.3f}'.format(np.mean(attr39_0)))
_ = plt.xlabel('Profit on Sales vs. Sales')
_ = plt.ylabel('Counts')
_ = plt.title('Distribution of Means of Profit on Sales vs. Sales\nfor Non-bankrupt Companies')
plt.show()

_ = plt.hist(attr39_1, bins=30)
_ = plt.axvline(x=np.mean(attr39_0), color='black', linestyle='dashed')
_ = plt.text(-0.5,100,'Mean = {:.3f}'.format(np.mean(attr39_1)))
_ = plt.xlabel('Profit on Sales vs. Sales')
_ = plt.ylabel('Counts')
_ = plt.title('Distribution of Means of Profit on Sales vs. Sales\nfor Bankrupt Companies')
plt.show()

means = (np.mean(attr39_0), np.mean(attr39_1), np.mean(df_NO.Attr39)) 
diff_means = np.abs(means[0] - means[1])

attr39_0_shifted = attr39_0 - means[0] + means[2]
attr39_1_shifted = attr39_1 - means[1] + means[2]

def bootstrap_replicates(array1, array2, size):
    """Build bootstrap replicates for two arrays"""
    
    bs_means_array1 = np.empty(size)
    bs_means_array2 = np.empty(size)
    
    for i in range(size):
        bs_array1 = np.random.choice(array1)
        bs_array2 = np.random.choice(array2)
        bs_means_array1[i], bs_means_array2[i] = np.mean(bs_array1), np.mean(bs_array2)
        
    return bs_means_array1, bs_means_array2

bs_attr39_0, bs_attr39_1 = bootstrap_replicates(attr39_0_shifted, attr39_1_shifted, 10000)
bs_diff_means = np.abs(bs_attr39_0 - bs_attr39_1)
_ = plt.hist(bs_diff_means, bins=30, color='green')
_ = plt.axvline(x=diff_means, color='black', linestyle='dashed')
_ = plt.text(0.19,2000,'Experimental Mean = {:.3f}'.format(diff_means))
_ = plt.xlabel('Difference of Means')
_ = plt.ylabel('Counts')
_ = plt.title('Difference of Means of Profit on Sales vs. Sales\nfor Both Types of Companies')
plt.savefig('Bootstrapped_Difference_of_Means.png')
plt.show()

p_value = np.sum(bs_diff_means >= diff_means)/len(bs_diff_means)

In [ ]:
# Machine Learning Setup Stage

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

X = df_NO.drop(['Target'], axis=1) #Dependent Variables
y = df_NO['Target'].astype('int64')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Implement Variable Inflation factor

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

final_iv, IV = data_vars(X_train, y_train)

features = list(IV[(IV['IV'] >= 0.01) & (IV['IV'] <= 0.8)]['VAR_NAME'])
X2 = X_train[features]

def iterate_vif(df, vif_threshold=5, max_vif=6):
  count = 0
  while max_vif > vif_threshold:
    count += 1
    print("Iteration # "+str(count))
    vif = pd.DataFrame()
    vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif["features"] = df.columns
    
    if vif['VIFactor'].max() > vif_threshold:
      print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], vif['VIFactor'].max()))
      df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
      max_vif = vif['VIFactor'].max()
    else:
        print('Complete')
        return df, vif.sort_values('VIFactor')
    
X1 = X2._get_numeric_data()
final_df, final_vif = iterate_vif(X1)

X_train = final_df
X_test=X_test[X_train.columns]

In [ ]:
#Dataframe of Compressed Feature Definitions with VIFs less than 5
fin_def_vif = pd.DataFrame(index=X_train.columns)
fin_def_vif = pd.merge(fin_def_vif, finance_def_df, left_index=True, right_index=True)

In [ ]:
#(Optional) Creating model with reduced features that are statistically significant

# df3-2: Compressed features by eliminating those that are not statistically significant
X_train = X_train[['Attr22','Attr24','Attr25','Attr3','Attr48','Attr49','Attr55','Attr56','Attr62']]
X_test=X_test[X_train.columns]

Choose a machine Learning Algorithm

In [ ]:
#1 Logistic Regression

def run_regression_accuracy(X_train, y_train, X_test, y_test):
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    cm = confusion_matrix(y_test, y_pred)
    print('\nConfusion matrix: \n',cm)

    print('\nClassification report: \n',classification_report(y_test, y_pred))

    logit_roc_auc = roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])

    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
#    plt.savefig('LogReg_ROC', dpi=300)
    plt.show()
  
    return logreg, fpr, tpr, thresholds

logreg, fpr, tpr, thresholds = run_regression_accuracy(X_train, y_train, X_test, y_test)

In [ ]:
#Logistic regression Summary

import statsmodels.api as sm
sm_model = sm.Logit(y_train, sm.add_constant(X_train)).fit()
sm_model.summary()

In [ ]:
# Hyperparameter Tuning for XGBoost and LightGBM

def GridSearchCV_Tuning(param_grid, classifier):
    
    model_class = classifier
    model_class_cv = GridSearchCV(model_class, param_grid, cv=10)
    
    model_class_cv.fit(X_train, y_train)
    
    print('\nBest Parameters: \n', model_class_cv.best_params_)
    print('\nBest Score: \n', model_class_cv.best_score_)
    
# Different classifiers: xgb.XGBClassifier(), lgb.LGBMClassifier()

param_grid = {'max_depth':np.arange(4,10), 'learning_rate':[0.0001, 0.001, 0.01, 0.1], \
              'n_estimators':[10, 20, 30, 40, 50]}

GridSearchCV_Tuning(param_grid, lgb.LGBMClassifier())

In [ ]:
#2 XGBoost

def XGBoost_accuracy(X_train, X_test, y_train, y_test):
    
    #df1: learning_rate=0.1, max_depth=4, n_estimators=50, seed=42
    #df2: learning_rate=0.1, max_depth=7, n_estimators=50, seed=42
    #df3-1: learning_rate=0.1, max_depth=4, n_estimators=40, seed=42
    #df3-2: learning_rate=0.1, max_depth=5, n_estimators=40, seed=42
    xgbclass = xgb.XGBClassifier(learning_rate=0.1, max_depth=4 , objective='binary:logistic', \
                             n_estimators=40, seed=42)
    xgbclass.fit(X_train, y_train)
    y_pred = xgbclass.predict(X_test)
    
    accuracy = float(np.sum(y_pred==y_test))/y_test.shape[0]
    print('\nAccuracy is {:.2f}\n'.format(accuracy))
    print('\nConfusion Matrix: \n',confusion_matrix(y_test, y_pred))
    print('\nClassification Report: \n',classification_report(y_test, y_pred))
    
    fpr1, tpr1, thresholds = roc_curve(y_test, xgbclass.predict_proba(X_test)[:,1])
    auc_score1 = roc_auc_score(y_test, xgbclass.predict_proba(X_test)[:,1])
    plt.plot(fpr1, tpr1, color='orange', label='XGBoost (area = %0.3f)' % auc_score1)
    plt.plot([0, 1],[0, 1], color='red', ls='--')
    plt.xlim([0,1])
    plt.ylim([0,1.05])
    plt.legend(loc='lower right')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True positive Rate')
    plt.title('Receiver Operating Characteristic')
#    plt.savefig('XGBoost_ROC', dpi=300)
    plt.show()
        
    return xgbclass, fpr1, tpr1

xgbclass, fpr1, tpr1 = XGBoost_accuracy(X_train, X_test, y_train, y_test)

In [ ]:
#3 LightGBM

def LightGBM_accuracy(X_train, X_test, y_train, y_test):
    
    #df1: learning_rate=0.1, max_depth=4, n_estimators=50, seed=42
    #df2: learning_rate=0.1, max_depth=9, n_estimators=40, seed=42
    #df3-1: learning_rate=0.1, max_depth=5, n_estimators=40, seed=42
    #df3-2: learning_rate=0.1, max_depth=6, n_estimators=30, seed=42
    lgbmclass = lgb.LGBMClassifier(learning_rate=0.1, max_depth=6, objective='binary', \
                             n_estimators=30, seed=42)
    lgbmclass.fit(X_train, y_train)
    y_pred = lgbmclass.predict(X_test)
    
    accuracy = float(np.sum(y_pred==y_test))/y_test.shape[0]
    print('\nAccuracy is {:.2f}\n'.format(accuracy))
    print('\nConfusion Matrix: \n',confusion_matrix(y_test, y_pred))
    print('\nClassification Report: \n',classification_report(y_test, y_pred))
    
    fpr2, tpr2, thresholds = roc_curve(y_test, lgbmclass.predict_proba(X_test)[:,1])
    auc_score2 = roc_auc_score(y_test, lgbmclass.predict_proba(X_test)[:,1])
    plt.plot(fpr2, tpr2, color='green', label='LightGBM (area = %0.3f)' % auc_score2)
    plt.plot([0, 1],[0, 1], color='red', ls='--')
    plt.xlim([0,1])
    plt.ylim([0,1.05])
    plt.legend(loc='lower right')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True positive Rate')
    plt.title('Receiver Operating Characteristic')
#    plt.savefig('LightGBM_ROC', dpi=300)
    plt.show()
        
    return lgbmclass, fpr2, tpr2

lgbmclass, fpr2, tpr2 = LightGBM_accuracy(X_train, X_test, y_train, y_test)